## Pre-processing
- One-hot encode categorical variables.
- Train/test split your data.
- Scale your data.
- Consider using automated feature selection.

## Modeling
- **Establish your baseline score.**
- Fit linear regression. Look at your coefficients. Are any of them wildly overblown?
- Fit lasso/ridge/elastic net with default parameters.
- Go back and remove features that might be causing issues in your models.
- Tune hyperparameters.
- **Identify a production model.** (This does not have to be your best performing Kaggle model, but rather the model that best answers your problem statement.)
- Refine and interpret your production model.


In [22]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, Normalizer, PolynomialFeatures 

import statsmodels.api as sm
import math


In [23]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [24]:
housing = pd.read_csv('./housing.csv', low_memory=False)
housing.head()

,Unnamed: 0,resale_price,Tranc_Year,Tranc_Month,full_flat_type,commercial,total_dwelling_units,planning_area,Mall_Within_2km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,pri_sch_name,vacancy,sec_sch_name,region,bus_mrt_interchange,est_floor_level,age_at_sale,resale_price_log,Mall_Nearest_Distance_log,Hawker_Nearest_Distance_log,mrt_nearest_distance_log,bus_stop_nearest_distance_log,pri_sch_nearest_distance_log,sec_sch_nearest_dist_log
0,0,680000.0,2016,5,4 ROOM Model A,N,142,Kallang,7.0,13.0,84,60,Geylang Methodist School,78,Geylang Methodist School,Central,no interchange,11,10,13.429848,6.997679,5.041833,5.799344,3.381926,7.037584,7.037584
1,1,665000.0,2012,7,5 ROOM Improved,N,112,Bishan,3.0,7.0,80,77,Kuo Chuan Presbyterian Primary School,45,Kuo Chuan Presbyterian Secondary School,Central,bus and mrt interchange,8,25,13.407542,6.764971,6.461706,6.806453,4.064019,6.029741,6.104557
2,3,550000.0,2012,4,4 ROOM Model A,Y,75,Bishan,4.0,9.0,32,86,Catholic High School,20,Catholic High School,Central,bus and mrt interchange,3,20,13.217674,6.856646,6.587846,6.810642,3.770379,5.964904,5.964904
3,4,298000.0,2017,12,4 ROOM Simplified,N,48,Yishun,2.0,1.0,45,0,Naval Base Primary School,74,Orchid Park Secondary School,North,no interchange,2,30,12.604849,6.592732,7.339636,6.021856,4.863084,5.994462,5.743085
4,5,335000.0,2013,1,3 ROOM Improved,Y,193,Geylang,6.0,11.0,79,82,Saint Margaret's Primary School,65,Geylang Methodist School,Central,mrt interchange,8,38,12.721886,6.527964,5.000034,6.519577,5.436689,6.387096,6.411553


In [25]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144889 entries, 0 to 144888
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     144889 non-null  int64  
 1   resale_price                   144889 non-null  float64
 2   Tranc_Year                     144889 non-null  int64  
 3   Tranc_Month                    144889 non-null  int64  
 4   full_flat_type                 144889 non-null  object 
 5   commercial                     144889 non-null  object 
 6   total_dwelling_units           144889 non-null  int64  
 7   planning_area                  144889 non-null  object 
 8   Mall_Within_2km                144889 non-null  float64
 9   Hawker_Within_2km              144889 non-null  float64
 10  hawker_food_stalls             144889 non-null  int64  
 11  hawker_market_stalls           144889 non-null  int64  
 12  pri_sch_name                  

In [26]:
# Converting the following variables to categorical 
# To see if transactions in a specific year or month impacts the resale price 

housing['Tranc_Year'] = housing['Tranc_Year'].astype('object')
housing['Tranc_Month'] = housing['Tranc_Month'].astype('object')

In [27]:
categorical_var = housing.select_dtypes(include=['object'])
categorical_var.columns

Index(['Tranc_Year', 'Tranc_Month', 'full_flat_type', 'commercial',
       'planning_area', 'pri_sch_name', 'sec_sch_name', 'region',
       'bus_mrt_interchange'],
      dtype='object')

### Pre-Processing

In [28]:
# ONE-HOT ENCODING CATEGORICAL VARIABLES

housing = pd.get_dummies(columns=categorical_var.columns, 
                         data=housing, drop_first=True)

/var/folders/r9/9246nqp9249ghjl6r9xy_dwr0000gn/T/ipykernel_46983/4120221262.py:3: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  housing = pd.get_dummies(columns=categorical_var.columns,


In [29]:
housing.head()

,Unnamed: 0,resale_price,total_dwelling_units,Mall_Within_2km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,vacancy,est_floor_level,age_at_sale,resale_price_log,Mall_Nearest_Distance_log,Hawker_Nearest_Distance_log,mrt_nearest_distance_log,bus_stop_nearest_distance_log,pri_sch_nearest_distance_log,sec_sch_nearest_dist_log,Tranc_Year_2013,Tranc_Year_2014,Tranc_Year_2015,Tranc_Year_2016,Tranc_Year_2017,Tranc_Year_2018,Tranc_Year_2019,Tranc_Year_2020,Tranc_Year_2021,Tranc_Month_2,Tranc_Month_3,Tranc_Month_4,Tranc_Month_5,Tranc_Month_6,Tranc_Month_7,Tranc_Month_8,Tranc_Month_9,Tranc_Month_10,Tranc_Month_11,Tranc_Month_12,full_flat_type_2 ROOM 2-room,full_flat_type_2 ROOM DBSS,full_flat_type_2 ROOM Improved,full_flat_type_2 ROOM Model A,full_flat_type_2 ROOM Premium Apartment,full_flat_type_2 ROOM Standard,full_flat_type_3 ROOM DBSS,full_flat_type_3 ROOM Improved,full_flat_type_3 ROOM Model A,full_flat_type_3 ROOM New Generation,full_flat_type_3 ROOM Premium Apartment,full_flat_type_3 ROOM Simplified,full_flat_type_3 ROOM Standard,...,sec_sch_name_Punggol Secondary School,sec_sch_name_Queenstown Secondary School,sec_sch_name_Queensway Secondary School,sec_sch_name_Raffles Girls' School,sec_sch_name_Raffles Institution,sec_sch_name_Regent Secondary School,sec_sch_name_River Valley High School,sec_sch_name_Riverside Secondary School,sec_sch_name_Saint Andrew's Secondary School,sec_sch_name_Saint Anthony's Canossian Secondary School,sec_sch_name_Saint Gabriel's Secondary School,sec_sch_name_Saint Hilda's Secondary School,sec_sch_name_Saint Margaret's Secondary School,sec_sch_name_Saint Patrick's School,sec_sch_name_Sembawang Secondary School,sec_sch_name_Seng Kang Secondary School,sec_sch_name_Serangoon Garden Secondary School,sec_sch_name_Serangoon Secondary School,sec_sch_name_Springfield Secondary School,sec_sch_name_Swiss Cottage Secondary School,sec_sch_name_Tampines Secondary School,sec_sch_name_Tanglin Secondary School,sec_sch_name_Tanjong Katong Secondary School,sec_sch_name_Teck Whye Secondary School,sec_sch_name_Temasek Junior College,sec_sch_name_Temasek Secondary School,sec_sch_name_Unity Secondary School,sec_sch_name_West Spring Secondary School,sec_sch_name_Westwood Secondary School,sec_sch_name_Whitley Secondary School,sec_sch_name_Woodgrove Secondary School,sec_sch_name_Woodlands Ring Secondary School,sec_sch_name_Woodlands Secondary School,sec_sch_name_Xinmin Secondary School,sec_sch_name_Yio Chu Kang Secondary School,sec_sch_name_Yishun Secondary School,sec_sch_name_Yishun Town Secondary School,sec_sch_name_Yuan Ching Secondary School,sec_sch_name_Yuhua Secondary School,sec_sch_name_Yusof Ishak Secondary School,sec_sch_name_Yuying Secondary School,sec_sch_name_Zhenghua Secondary School,sec_sch_name_Zhonghua Secondary School,region_East,region_North,region_North-East,region_West,bus_mrt_interchange_bus interchange,bus_mrt_interchange_mrt interchange,bus_mrt_interchange_no interchange
0,0,680000.0,142,7.0,13.0,84,60,78,11,10,13.429848,6.997679,5.041833,5.799344,3.381926,7.037584,7.037584,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,665000.0,112,3.0,7.0,80,77,45,8,25,13.407542,6.764971,6.461706,6.806453,4.064019,6.029741,6.104557,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,550000.0,75,4.0,9.0,32,86,20,3,20,13.217674,6.856646,6.587846,6.810642,3.770379,5.964904,5.964904,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,298000.0,48,2.0,1.0,45,0,74,2,30,12.604849,6.592732,7.339636,6.021856,4.863084,5.994462,5.743085,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
housing.isnull().sum()

Unnamed: 0                             0
resale_price                           0
total_dwelling_units                   0
Mall_Within_2km                        0
Hawker_Within_2km                      0
                                      ..
region_North-East                      0
region_West                            0
bus_mrt_interchange_bus interchange    0
bus_mrt_interchange_mrt interchange    0
bus_mrt_interchange_no interchange     0
Length: 424, dtype: int64

In [31]:
housing.drop(columns = "Unnamed: 0", inplace = True)

In [32]:
housing.describe(include = 'all')

,resale_price,total_dwelling_units,Mall_Within_2km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,vacancy,est_floor_level,age_at_sale,resale_price_log,Mall_Nearest_Distance_log,Hawker_Nearest_Distance_log,mrt_nearest_distance_log,bus_stop_nearest_distance_log,pri_sch_nearest_distance_log,sec_sch_nearest_dist_log,Tranc_Year_2013,Tranc_Year_2014,Tranc_Year_2015,Tranc_Year_2016,Tranc_Year_2017,Tranc_Year_2018,Tranc_Year_2019,Tranc_Year_2020,Tranc_Year_2021,Tranc_Month_2,Tranc_Month_3,Tranc_Month_4,Tranc_Month_5,Tranc_Month_6,Tranc_Month_7,Tranc_Month_8,Tranc_Month_9,Tranc_Month_10,Tranc_Month_11,Tranc_Month_12,full_flat_type_2 ROOM 2-room,full_flat_type_2 ROOM DBSS,full_flat_type_2 ROOM Improved,full_flat_type_2 ROOM Model A,full_flat_type_2 ROOM Premium Apartment,full_flat_type_2 ROOM Standard,full_flat_type_3 ROOM DBSS,full_flat_type_3 ROOM Improved,full_flat_type_3 ROOM Model A,full_flat_type_3 ROOM New Generation,full_flat_type_3 ROOM Premium Apartment,full_flat_type_3 ROOM Simplified,full_flat_type_3 ROOM Standard,full_flat_type_3 ROOM Terrace,...,sec_sch_name_Punggol Secondary School,sec_sch_name_Queenstown Secondary School,sec_sch_name_Queensway Secondary School,sec_sch_name_Raffles Girls' School,sec_sch_name_Raffles Institution,sec_sch_name_Regent Secondary School,sec_sch_name_River Valley High School,sec_sch_name_Riverside Secondary School,sec_sch_name_Saint Andrew's Secondary School,sec_sch_name_Saint Anthony's Canossian Secondary School,sec_sch_name_Saint Gabriel's Secondary School,sec_sch_name_Saint Hilda's Secondary School,sec_sch_name_Saint Margaret's Secondary School,sec_sch_name_Saint Patrick's School,sec_sch_name_Sembawang Secondary School,sec_sch_name_Seng Kang Secondary School,sec_sch_name_Serangoon Garden Secondary School,sec_sch_name_Serangoon Secondary School,sec_sch_name_Springfield Secondary School,sec_sch_name_Swiss Cottage Secondary School,sec_sch_name_Tampines Secondary School,sec_sch_name_Tanglin Secondary School,sec_sch_name_Tanjong Katong Secondary School,sec_sch_name_Teck Whye Secondary School,sec_sch_name_Temasek Junior College,sec_sch_name_Temasek Secondary School,sec_sch_name_Unity Secondary School,sec_sch_name_West Spring Secondary School,sec_sch_name_Westwood Secondary School,sec_sch_name_Whitley Secondary School,sec_sch_name_Woodgrove Secondary School,sec_sch_name_Woodlands Ring Secondary School,sec_sch_name_Woodlands Secondary School,sec_sch_name_Xinmin Secondary School,sec_sch_name_Yio Chu Kang Secondary School,sec_sch_name_Yishun Secondary School,sec_sch_name_Yishun Town Secondary School,sec_sch_name_Yuan Ching Secondary School,sec_sch_name_Yuhua Secondary School,sec_sch_name_Yusof Ishak Secondary School,sec_sch_name_Yuying Secondary School,sec_sch_name_Zhenghua Secondary School,sec_sch_name_Zhonghua Secondary School,region_East,region_North,region_North-East,region_West,bus_mrt_interchange_bus interchange,bus_mrt_interchange_mrt interchange,bus_mrt_interchange_no interchange
count,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,...,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,144889.000000,14488

In [33]:
housing.shape

(144889, 423)

### Train Test Split

In [34]:
X = housing.drop(columns = ['resale_price','resale_price_log'])
y = housing['resale_price_log']
y_orig = housing['resale_price']

In [35]:
# # Instantiate our PolynomialFeatures object to create all two-way terms.
# poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# # Fit and transform our X data.
# X_overfit = poly.fit_transform(X)

In [36]:
#poly.get_feature_names_out(X.columns)

In [37]:
# Baseline model 
X_baseline = sm.add_constant(X)
ols = sm.OLS(y, X_baseline).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       resale_price_log   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     3762.
Date:                Sat, 28 Jan 2023   Prob (F-statistic):               0.00
Time:                        21:31:22   Log-Likelihood:             1.5595e+05
No. Observations:              144889   AIC:                        -3.111e+05
Df Residuals:                  144493   BIC:                        -3.072e+05
Df Model:                         395                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
const                                                      11.4927      0.013    907.006      0.000      11.468      11.518
total_dwelling_units                                       -0.0001    5.3e-06    -22.086      0.000      -0.000      -0.000
Mall_Within_2km                                             0.0011      0.000      5.877      0.000       0.001       0.002
Hawker_Within_2km                                           0.0034      0.000      9.358      0.000       0.003       0.004
hawker_food_stalls                                      -9.558e-05   2.41e-05     -3.958      0.000      -0.000   -4.82e-05
hawker_market_stalls                                        0.0001   9.66e-06     10.521      0.000    8.27e-05       0.000
vacancy                                                     0.0101      0.000     61.708      0.000       0.010       0.010
est_floor_level                                             0.0070   4.82e-05    146.311      0.000       0.007       0.007
age_at_sale                                                -0.0065   4.31e-05   -149.681      0.000      -0.007      -0.006
Mall_Nearest_Distance_log                                  -0.0249      0.001    -37.511      0.000      -0.026      -0.024
Hawker_Nearest_Distance_log                                -0.0069      0.001    -10.278      0.000      -0.008      -0.006
mrt_nearest_distance_log                                   -0.0599      0.001    -81.724      0.000      -0.061      -0.058
bus_stop_nearest_distance_log                              -0.0035      0.000     -7.824      0.000      -0.004      -0.003
pri_sch_nearest_distance_log                                0.0018      0.001      3.365      0.001       0.001       0.003
sec_sch_nearest_dist_log                                    0.0002      0.001      0.419      0.675      -0.001       0.001
Tranc_Year_2013                                             0.0315      0.001     31.826      0.000       0.030       0.033
Tranc_Year_2014                                            -0.0313      0.001    -31.492      0.000      -0.033      -0.029
Tranc_Year_2015                                            -0.0746      0.001    -76.657      0.000      -0.077      -0.073
Tranc_Year_2016                                            -0.0755      0.001    -78.727      0.000      -0.077      -0.074
Tranc_Year_2017                                            -0.0877      0.001    -92.081      0.000      -0.090      -0.086
Tranc_Year_2018                                            -0.1037      0.001   -108.960      0.000      -0.106      -0.102
Tranc_Year_2019                                            -0.1087      0.001   -114.336      0.000      -0

### Linear Regression

In [38]:
# Train-test-split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

In [39]:
# Standardising predictors

ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [40]:
# Fitting the model
lr_model = LinearRegression()
lr_model.fit(Z_train,y_train)

LinearRegression()

In [41]:
# Returning the R^2 for the model

#Training set R2
print('Training set R^2: ', lr_model.score(Z_train, y_train))

#Test set R2
print('Test set R^2: ', lr_model.score(Z_test, y_test))

Training set R^2:  0.9108053936215674
Test set R^2:  0.9120471566158838


In [42]:
lr_model_scores = cross_val_score(lr_model, Z_train, y_train, cv=10)

print (lr_model_scores)
print (np.mean(lr_model_scores))

[ 9.12758778e-01  9.10901180e-01  9.10612439e-01 -1.98759457e+23
  9.08025281e-01  9.09118708e-01  9.08397098e-01 -6.51602989e+23
  9.10598487e-01  9.07266028e-01]
-8.503624462365595e+22


In [43]:
# Model does not seem to have overfit.

In [44]:
Z_train_lr = sm.add_constant(Z_train)
ols = sm.OLS(y_train, Z_train_lr).fit()
ols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       resale_price_log   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                     681.5
Date:                Sat, 28 Jan 2023   Prob (F-statistic):               0.00
Time:                        21:32:41   Log-Likelihood:                 52158.
No. Observations:              101422   AIC:                        -1.035e+05
Df Residuals:                  101026   BIC:                        -9.975e+04
Df Model:                         395                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.9441      0.000   2.84e+04      0.000      12.943      12.945
x1            -0.0068      0.001    -10.192      0.000      -0.008      -0.005
x2             0.0030      0.001      2.159      0.031       0.000       0.006
x3             0.0147      0.003      4.896      0.000       0.009       0.021
x4            -0.0017      0.001     -1.816      0.069      -0.004       0.000
x5             0.0067      0.001      5.866      0.000       0.004       0.009
x6          6.868e+09   7.36e+10      0.093      0.926   -1.37e+11    1.51e+11
x7             0.0352      0.001     69.100      0.000       0.034       0.036
x8            -0.0765      0.001    -71.438      0.000      -0.079      -0.074
x9            -0.0153      0.001    -18.032      0.000      -0.017      -0.014
x10           -0.0068      0.001     -4.937      0.000      -0.009      -0.004
x11           -0.0359      0.001    -39.212      0.000      -0.038      -0.034
x12           -0.0018      0.000     -3.725      0.000      -0.003      -0.001
x13            0.0008      0.001      1.259      0.208      -0.000       0.002
x14         8.845e-05      0.001      0.128      0.898      -0.001       0.001
x15            0.0088      0.001     14.944      0.000       0.008       0.010
x16           -0.0091      0.001    -15.454      0.000      -0.010      -0.008
x17           -0.0219      0.001    -36.274      0.000      -0.023      -0.021
x18           -0.0230      0.001    -37.416      0.000      -0.024      -0.022
x19           -0.0274      0.001    -43.491      0.000      -0.029      -0.026
x20           -0.0331      0.001    -51.610      0.000      -0.034      -0.032
x21           -0.0353      0.001    -54.352      0.000      -0.037      -0.034
x22           -0.0264      0.001    -39.389      0.000      -0.028      -0.025
x23           -0.0040      0.001     -6.985      0.000      -0.005      -0.003
x24            0.0005      0.001      0.770      0.441      -0.001       0.002
x25           -0.0002      0.001     -0.254      0.800      -0.001       0.001
x26            0.0004      0.001      0.622      0.534      -0.001       0.002
x27           -0.0004      0.001     -0.580      0.562      -0.002       0.001
x28           -0.0014      0.001     -2.153      0.031      -0.003      -0.000
x29           -0.0007      0.001     -1.104      0.270      -0.002       0.001
x30           -0.0008      0.001     -1.224      0.221      -0.002       0.000
x31           -0.0010      0.001     -1.456      0.145      -0.002       0.000
x32           -0.0003      0.001     -0.402      0.688      -0.002       0.001
x33           -0.0002      0.001     -0.266      0.790      -0.001       0.001
x34         6.076e-05      0.001      0.095      0.924      -0.001       0.001
x35            0.0017      0.000      3.584      0.000       0.001       0.003
x3

In [45]:
# Function for obtaining coefficient names, values and p-values from OLS model:

def get_coef_table(lin_reg, variable):
    err_series = lin_reg.params - lin_reg.conf_int()[0]
    
    coef_df = pd.DataFrame({'varname': variable.columns,
                            'coef': lin_reg.params.values[1:],
                            'ci_err': err_series.values[1:],
                            'pvalue': lin_reg.pvalues.round(4).values[1:]
                           })
    return coef_df

In [46]:
# display all the  rows
#pd.set_option('display.max_rows', None)

get_coef_table(ols, X)

,varname,coef,ci_err,pvalue
0,total_dwelling_units,-6.774339e-03,1.302744e-03,0.0000
1,Mall_Within_2km,2.970926e-03,2.696698e-03,0.0308
2,Hawker_Within_2km,1.474561e-02,5.902596e-03,0.0000
3,hawker_food_stalls,-1.735010e-03,1.872787e-03,0.0694
4,hawker_market_stalls,6.690893e-03,2.235511e-03,0.0000
5,vacancy,6.867841e+09,1.442416e+11,0.9256
6,est_floor_level,3.518675e-02,9.980509e-04,0.0000
7,age_at_sale,-7.647699e-02,2.098248e-03,0.0000
8,Mall_Nearest_Distance_log,-1.525825e-02,1.658509e-03,0.0000
9,Hawker_Nearest_Distance_log,-6.772136e-03,2.688693e-03,0.0000


In [47]:
# Predictions using LR
lr_y_pred = lr_model.predict(Z_test)

In [48]:
print("Linear Regression Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test,lr_y_pred)))

Linear Regression Root Mean Squared Error: 0.0822195711704881


### Ridge Regression

In [49]:
# finding the optimal alpha 

ridge_alphas = np.logspace(0, 5, 200)

optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=10)
optimal_ridge.fit(Z_train, y_train)

print (optimal_ridge.alpha_)

2.1214517849106302


In [50]:
# fitting to the model and getting R^2 scores

ridge = Ridge(alpha=optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, Z_train, y_train, cv=10)

print ('Cross-Validation scores:', ridge_scores)
print ('Mean Cross-Validation score:', np.mean(ridge_scores))

Cross-Validation scores: [0.91275824 0.91094783 0.91057477 0.91229046 0.9081103  0.90920118
 0.90851984 0.90800724 0.91069506 0.90730888]
Mean Cross-Validation score: 0.9098413803868288


In [51]:
ridge.fit(Z_train, y_train)

Ridge(alpha=2.1214517849106302)

In [52]:
print("Training score:", ridge.score(Z_train, y_train))
print("Test score:", ridge.score(Z_test, y_test))

Training score: 0.9108872562080264
Test score: 0.9120983993709808


In [53]:
# Predictions using Ridge
ridge_y_pred = ridge.predict(Z_test)
pd.DataFrame(ridge_y_pred).head()

,0
0,12.921904
1,13.169668
2,13.002454
3,12.574117
4,13.084085


In [54]:
print("Ridge Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test,ridge_y_pred)))

Ridge Root Mean Squared Error: 0.08219561645208533


In [55]:
# Function for getting dollar increase in resale price for 1 unit change in predictor 

def coef_fx(model):
    transformed_coef = []
    for i in model.coef_:
        j = math.exp(i)
        transformed_coef.append(j)
        coef_effect = [(i-1)*housing['resale_price'].mean() for i in transformed_coef]
    return coef_effect

In [56]:
coef_fx(ridge)[:5]

[-2959.228834400788,
 1286.0083341006225,
 6426.515810296233,
 -769.2865596558122,
 2932.4894649979874]

In [57]:
# Summarising coefficients

ridge_coefs = pd.DataFrame({'variable':X.columns,
                            'coef':ridge.coef_,
                            'abs_coef':np.abs(ridge.coef_),
                            'coef_effect':coef_fx(ridge),
                           })

ridge_coefs.sort_values('abs_coef', inplace=True, ascending=False)
ridge_coefs.head(50)

,variable,coef,abs_coef,coef_effect
60,full_flat_type_5 ROOM Improved,0.360192,0.360192,188365.328888
51,full_flat_type_4 ROOM Model A,0.339219,0.339219,175439.394969
68,full_flat_type_EXECUTIVE Apartment,0.232469,0.232469,113691.600124
69,full_flat_type_EXECUTIVE Maisonette,0.193514,0.193514,92750.573950
64,full_flat_type_5 ROOM Premium Apartment,0.186632,0.186632,89135.028812
54,full_flat_type_4 ROOM Premium Apartment,0.186011,0.186011,88810.140143
53,full_flat_type_4 ROOM New Generation,0.170292,0.170292,80649.752647
43,full_flat_type_3 ROOM New Generation,0.144697,0.144697,67633.819818
55,full_flat_type_4 ROOM Simplified,0.124340,0.124340,57517.066510
42,full_flat_type_3 ROOM Model A,0.124246,0.124246,57470.929985


### Lasso Regression

In [58]:
optimal_lasso = LassoCV(n_alphas=1000, cv=10)
optimal_lasso.fit(Z_train, y_train)

print (optimal_lasso.alpha_)

0.0001138842801659491


/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.465e+00, tolerance: 7.781e-01
  model = cd_fast.enet_coordinate_descent(


In [59]:
lasso = Lasso(alpha=optimal_lasso.alpha_)
lasso_scores = cross_val_score(lasso, Z_train, y_train, cv=10)

print ('Cross-Validation scores:', lasso_scores)
print ('Cross-Validation mean score:', np.mean(lasso_scores))

/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.547e+00, tolerance: 7.002e-01
  model = cd_fast.enet_coordinate_descent(
/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.205e+00, tolerance: 6.998e-01
  model = cd_fast.enet_coordinate_descent(
/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

Cross-Validation scores: [0.91050912 0.90765143 0.90722759 0.9111551  0.90630116 0.90657687
 0.90753559 0.90740342 0.90826848 0.90498302]
Cross-Validation mean score: 0.9077611774743432


/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.365e+00, tolerance: 7.002e-01
  model = cd_fast.enet_coordinate_descent(


In [60]:
lasso.fit(Z_train, y_train)

/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.465e+00, tolerance: 7.781e-01
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.0001138842801659491)

In [61]:
print("Training set score:", lasso.score(Z_train, y_train))
print("Test set score:", lasso.score(Z_test, y_test))

Training set score: 0.9086183196842404
Test set score: 0.909507383552211


In [62]:
lasso_coefs = pd.DataFrame({'variable':X.columns,
                            'coef':lasso.coef_,
                            'abs_coef':np.abs(lasso.coef_),
                            'coef_effect':coef_fx(lasso),
                           })

lasso_coefs.sort_values('abs_coef', inplace=True, ascending=False)
lasso_coefs.head(20)

,variable,coef,abs_coef,coef_effect
60,full_flat_type_5 ROOM Improved,0.164730,0.164730,77793.140320
68,full_flat_type_EXECUTIVE Apartment,0.130140,0.130140,60378.435348
51,full_flat_type_4 ROOM Model A,0.115862,0.115862,53364.074081
69,full_flat_type_EXECUTIVE Maisonette,0.110780,0.110780,50891.552181
64,full_flat_type_5 ROOM Premium Apartment,0.086293,0.086293,39152.187560
7,age_at_sale,-0.078154,0.078154,-32658.431046
54,full_flat_type_4 ROOM Premium Apartment,0.069125,0.069125,31091.188387
70,full_flat_type_EXECUTIVE Premium Apartment,0.062739,0.062739,28127.911356
62,full_flat_type_5 ROOM Model A,0.055604,0.055604,24839.322519
53,full_flat_type_4 ROOM New Generation,0.050414,0.050414,22461.955145


In [63]:
# Predictors that were removed
lasso_coefs[lasso_coefs['abs_coef'] == 0]

,variable,coef,abs_coef,coef_effect
104,planning_area_Yishun,0.0,0.0,0.0
389,sec_sch_name_Springfield Secondary School,-0.0,0.0,0.0
82,planning_area_Clementi,0.0,0.0,0.0
31,Tranc_Month_10,-0.0,0.0,0.0
216,pri_sch_name_Princess Elizabeth Primary School,-0.0,0.0,0.0
191,pri_sch_name_Meridian Primary School,0.0,0.0,0.0
215,pri_sch_name_Poi Ching School,0.0,0.0,0.0
113,pri_sch_name_Bedok Green Primary School,-0.0,0.0,0.0
76,planning_area_Bukit Batok,-0.0,0.0,0.0
258,pri_sch_name_Wellington Primary School,0.0,0.0,0.0


In [64]:
lasso_y_pred = lasso.predict(Z_test)
lasso_y_pred

array([12.91370476, 13.1690346 , 12.9970155 , ..., 12.97207371,
       12.8596006 , 12.64290829])

In [65]:
print("Lasso Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test,lasso_y_pred)))

Lasso Root Mean Squared Error: 0.08339823084893068


In [66]:
# Predicting on the test set

lasso_predicted = pd.DataFrame({'y_hat':lasso.predict(Z_test),
                               'y_actual': y_test,
                               'residuals': (y_test - lasso.predict(Z_test)),
                               'actual test values': np.exp(y_test),
                               'predicted values': np.exp(lasso_y_pred).round(decimals = 1)                             
                              })

lasso_predicted.sort_values('residuals', inplace=True, ascending=False)
lasso_predicted.head(10)

,y_hat,y_actual,residuals,actual test values,predicted values
52720,12.876973,13.428211,0.551238,678888.0,391199.6
110611,12.930470,13.458978,0.528508,700100.0,412697.6
116687,12.934104,13.459121,0.525017,700200.0,414199.9
74487,12.861573,13.377006,0.515432,645000.0,385221.4
102073,12.858590,13.366094,0.507504,638000.0,384073.6
53567,12.943051,13.429848,0.486797,680000.0,417922.4
45481,12.872432,13.345507,0.473075,625000.0,389427.1
141392,12.671742,13.142166,0.470424,510000.0,318616.2
46387,12.880123,13.337475,0.457352,620000.0,392433.6
8663,12.873317,13.329378,0.456060,615000.0,389772.0


### ElasticNet

In [67]:
# Finding the optimal alpha and l1 ratio

l1_ratios = np.linspace(0.01, 1.0, 25)

optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=1000, cv=10)
optimal_enet.fit(Z_train, y_train)

print (f'Optimal alpha: {optimal_enet.alpha_}')
print (f'Optimal L1 ratio: {optimal_enet.l1_ratio_}')


Optimal alpha: 0.0001138842801659491
Optimal L1 ratio: 1.0


/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.465e+00, tolerance: 7.781e-01
  model = cd_fast.enet_coordinate_descent(


In [68]:
# fitting the model

enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)

enet_scores = cross_val_score(enet, Z_train, y_train, cv=10)

print ("Cross-Validation scores: ", enet_scores)
print ("Cross-Validation mean score: ", np.mean(enet_scores))

/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.547e+00, tolerance: 7.002e-01
  model = cd_fast.enet_coordinate_descent(
/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.205e+00, tolerance: 6.998e-01
  model = cd_fast.enet_coordinate_descent(
/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

Cross-Validation scores:  [0.91050912 0.90765143 0.90722759 0.9111551  0.90630116 0.90657687
 0.90753559 0.90740342 0.90826848 0.90498302]
Cross-Validation mean score:  0.9077611774743432


/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.365e+00, tolerance: 7.002e-01
  model = cd_fast.enet_coordinate_descent(


In [69]:
enet.fit(Z_train, y_train)

/Users/kh/opt/anaconda3/envs/dsi-sg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.465e+00, tolerance: 7.781e-01
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.0001138842801659491, l1_ratio=1.0)

In [70]:
print("Training set scores: ", enet.score(Z_train, y_train))
print("Test set scores: ", enet.score(Z_test, y_test))

Training set scores:  0.9086183196842404
Test set scores:  0.909507383552211


In [71]:
enet_y_pred = enet.predict(Z_test)

In [72]:
print("ElasticNet Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test,enet_y_pred)))

ElasticNet Root Mean Squared Error: 0.08339823084893068


In [73]:
# Predicting on the test set

enet_predicted = pd.DataFrame({'y_hat':enet.predict(Z_test),
                               'y_actual': y_test,
                               'residuals': (y_test - enet.predict(Z_test)),
                               'actual test values': np.exp(y_test),
                               'predicted values': np.exp(enet_y_pred).round(decimals = 1)                             
                              })

enet_predicted.sort_values('residuals', inplace=True, ascending=False)
enet_predicted.head(10)

,y_hat,y_actual,residuals,actual test values,predicted values
52720,12.876973,13.428211,0.551238,678888.0,391199.6
110611,12.930470,13.458978,0.528508,700100.0,412697.6
116687,12.934104,13.459121,0.525017,700200.0,414199.9
74487,12.861573,13.377006,0.515432,645000.0,385221.4
102073,12.858590,13.366094,0.507504,638000.0,384073.6
53567,12.943051,13.429848,0.486797,680000.0,417922.4
45481,12.872432,13.345507,0.473075,625000.0,389427.1
141392,12.671742,13.142166,0.470424,510000.0,318616.2
46387,12.880123,13.337475,0.457352,620000.0,392433.6
8663,12.873317,13.329378,0.456060,615000.0,389772.0


In [74]:
# Coefficients of predictors

enet_coefs = pd.DataFrame({'variable':X.columns,
                           'enet_coef':enet.coef_,
                           'enet_abs_coef':np.abs(enet.coef_),
                           'coef_effect':coef_fx(enet)})

enet_coefs.sort_values('enet_abs_coef', inplace=True, ascending=False)
enet_coefs.head(30)

,variable,enet_coef,enet_abs_coef,coef_effect
60,full_flat_type_5 ROOM Improved,0.164730,0.164730,77793.140320
68,full_flat_type_EXECUTIVE Apartment,0.130140,0.130140,60378.435348
51,full_flat_type_4 ROOM Model A,0.115862,0.115862,53364.074081
69,full_flat_type_EXECUTIVE Maisonette,0.110780,0.110780,50891.552181
64,full_flat_type_5 ROOM Premium Apartment,0.086293,0.086293,39152.187560
7,age_at_sale,-0.078154,0.078154,-32658.431046
54,full_flat_type_4 ROOM Premium Apartment,0.069125,0.069125,31091.188387
70,full_flat_type_EXECUTIVE Premium Apartment,0.062739,0.062739,28127.911356
62,full_flat_type_5 ROOM Model A,0.055604,0.055604,24839.322519
53,full_flat_type_4 ROOM New Generation,0.050414,0.050414,22461.955145


---

### Review

**Comparison between the 3 Models**:

- All 3 models appear to not have overfitted, given the cross-validation scores.
- Ridge, Lasso and ElasticNet perform similarly in terms of R^2 scores (about 87%)
- Strongest predictors: floor area, age, distance to mrt and hawkers, whether the estate is in Woodlands and floor level

**Interpretation:**

<ol>
    <li> With every one unit increase in floor area, resale price is estimated to increase by SGD 85K </li>
    <li>  With every one unit increase in lease commence date, the resale price is estimated to increase by SGD47K</li>
    <li>Compared to houses in Kallang, resale prices in Woodlands, Jurong East, Sembawang, Choa Chu Kang are estimated to be SGD 20K - 36K lower, while prices in Marine Parade, Bishan, Bt Merah and Queenstown are estimated to be SGD 11k - 15k higher. </li>
    <li>With every 1 unit increase in distance from mrt and hawker centres, the resale price is estimated to decrease by SGD 25K and SGD 23K respectively.</li>
    <li>With every 1 unit increase in the estimated height of the house relative to the highest storey, the resale price is estimated to increase by SGD 15k.</li>
</ol>


**Conclusion and Recommendations**

- Houses with larger floor area, have longer remaining lease periods, which are closer to hawker centres and mrt stations, and are mature estates located closer to central area tend to fetch higher resale prices.
- On the flipside, houses located far away from the city centre have lower resale prices. 

- The results indicate that people value having a larger and newer home, which allows owners to have more flexibilty in terms of family planning or rental. 
- Accessibility to public transport (mrt stations) and cheap F&B options (hawkers) are important factors that impact day-to-day activities such as commuting and overall cost of living. 
- Keeping these needs and preferences in mind when attempting to market or sell a house would be useful, by emphasising on the property's strengths and downplaying its weaknesses. 

Future Steps
- TBC

